In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import torch
sys.path.append(os.path.abspath('..'))
from train_fsl import train, dataloaders, ppdf

In [3]:
data_dir = "/Users/pi/repos/gatech/8803-LS/Project/dataset/voxceleb2"
loaders = dataloaders(data_dir)

In [5]:
batch = next(iter(loaders['train']))

In [8]:
batch[0].shape

torch.Size([25, 1, 512, 300])

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [14]:
from audionet import AudioNet
from easyfsl.methods import RelationNetworks
N_WAY = 5 
backbone=AudioNet(N_WAY, mode='fe')
model = RelationNetworks(backbone)
model.to(device)

(4096, 1, 8)


RelationNetworks(
  (backbone): AudioNet(
    (features): Sequential(
      (conv1): Conv2d(1, 96, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
      (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.5, affine=True, track_running_stats=True)
      (relu1): ReLU()
      (mpool1): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.5, affine=True, track_running_stats=True)
      (relu2): ReLU()
      (mpool2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
      (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(384, eps=1e-05, momentum=0.5, affine=True, track_running_stats=True)
      (relu3): ReLU()
      (conv4): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn4): BatchNorm2d(256, eps=1e-05, momentum=0.5

In [15]:
train(model, loaders, device)

Start Training


Epoch [1/150]:   0%|                                                                                                                  | 0/100 [00:00<?, ?it/s][W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Epoch [1/150]:   0%|                                  

KeyboardInterrupt: 